In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
#!/usr/bin/env python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dateutil.parser
import datetime

## getting dataset
import os
from os import listdir

#########################
# configuration
#########################
data_path = 'drive/MyDrive/DS_Project_Datasets/Networkrail/Delay/'
file_list = listdir(data_path)
#file_list = ['2019_20_P01_all_delays.csv', '2021_22_P11_all_delays.csv', '2021_22_P09_all_delays.csv']

print(file_list)

['2019_20_P01_all_delays.csv', '2021_22_P11_all_delays.csv', '2021_22_P09_all_delays.csv', '2021_22_P06_all_delays.csv', '2022_23_P01_all_delays.csv', '2020_21_P13_all_delays.csv', '2018_19_P11_all_delays.csv', '2021_22_P03_all_delays.csv', '2018_19_P12_all_delays.csv', '2020_21_P06_all_delays.csv', '2018_19_P02_all_delays.csv', '2018_19_P07_all_delays.csv', '2019_20_P04_all_delays.csv', '2019_20_P02_all_delays.csv', '2019_20_P08_all_delays.csv', '2020_21_P02_all_delays.csv', '2019_20_P05_all_delays.csv', '2020_21_P04_all_delays.csv', '2019_20_P06_all_delays.csv', '2021_22_P05_all_delays.csv', '2019_20_P03_all_delays.csv', '2018_19_P05_all_delays.csv', '2021_22_P04_all_delays.csv', '2019_20_P10_all_delays.csv', '2021_22_P07_all_delays.csv', '2018_19_P09_all_delays.csv', '2018_19_P13_all_delays.csv', '2020_21_P01_all_delays.csv', '2021_22_P13_all_delays.csv', '2020_21_P11_all_delays.csv', '2018_19_P10_all_delays.csv', '2020_21_P08_all_delays.csv', '2020_21_P12_all_delays.csv', '2020_21_

In [ ]:
#get incident dict
incident_dict = pd.read_excel('drive/MyDrive/DS_Project_Datasets/Networkrail/Glossary/' + 'Historic-Delay-Attribution-Glossary (1).xlsx', sheet_name='Incident Reason').drop(labels=[365, 366], axis="index")[['Incident Reason','Incident JPIP Category']].set_index('Incident Reason',drop=True).to_dict()['Incident JPIP Category']

In [ ]:
for index, file in enumerate(file_list):
    #print(file)
    data = pd.read_csv(data_path + file, low_memory=False)

    #top 5 delay stations by start_stanox
    drop_col = data.columns.tolist()
    drop_col.remove('ORIGIN_DEPARTURE_DATE')
    drop_col.remove('INCIDENT_REASON')
    drop_col.remove('PFPI_MINUTES')

    tmp = data.drop(drop_col, axis = 1).groupby(by=['ORIGIN_DEPARTURE_DATE','INCIDENT_REASON']).apply(lambda s: pd.Series({ "count": s["INCIDENT_REASON"].count(), "minutes": s["PFPI_MINUTES"].sum()}))
    
    tmp = tmp.reset_index()
    tmp['ORIGIN_DEPARTURE_DATE'] = tmp['ORIGIN_DEPARTURE_DATE'].apply(lambda s: dateutil.parser.parse(s).date().strftime('%Y-%m-%d'))
    tmp['INCIDENT_REASON'] = tmp['INCIDENT_REASON'].apply(lambda s: incident_dict.get(s,'All Others'))
    tmp = tmp.groupby(by=['ORIGIN_DEPARTURE_DATE','INCIDENT_REASON']).apply(lambda s: pd.Series({"count_category": s["count"].sum(), "sum": s["minutes"].sum()}))
    
    if (index == 0):
      incident_data = tmp
    else:
      incident_data = incident_data.append(tmp)

    print('==> done ', str(index+1), "/", len(file_list))


==> done  1 / 53
==> done  2 / 53
==> done  3 / 53
==> done  4 / 53
==> done  5 / 53
==> done  6 / 53
==> done  7 / 53
==> done  8 / 53
==> done  9 / 53
==> done  10 / 53
==> done  11 / 53
==> done  12 / 53
==> done  13 / 53
==> done  14 / 53
==> done  15 / 53
==> done  16 / 53
==> done  17 / 53
==> done  18 / 53
==> done  19 / 53
==> done  20 / 53
==> done  21 / 53
==> done  22 / 53
==> done  23 / 53
==> done  24 / 53
==> done  25 / 53
==> done  26 / 53
==> done  27 / 53
==> done  28 / 53
==> done  29 / 53
==> done  30 / 53
==> done  31 / 53
==> done  32 / 53
==> done  33 / 53
==> done  34 / 53
==> done  35 / 53
==> done  36 / 53
==> done  37 / 53
==> done  38 / 53
==> done  39 / 53
==> done  40 / 53
==> done  41 / 53
==> done  42 / 53
==> done  43 / 53
==> done  44 / 53
==> done  45 / 53
==> done  46 / 53
==> done  47 / 53
==> done  48 / 53
==> done  49 / 53
==> done  50 / 53
==> done  51 / 53
==> done  52 / 53
==> done  53 / 53


In [ ]:
incident_data = incident_data.reset_index()

In [ ]:
#incident_data.to_csv('./drive/MyDrive/DS Project Notebook Files/DS_Project_Code/Jeremias/results/incident_reason_count.csv', header=True, index=False)

In [29]:
#incident_data = pd.read_csv('./drive/MyDrive/DS Project Notebook Files/DS_Project_Code/Jeremias/results/incident_reason_count.csv', low_memory=False)
#incident_data

In [45]:
a = incident_data.drop(['ORIGIN_DEPARTURE_DATE'], axis = 1).groupby(by=['INCIDENT_REASON']).apply(lambda s: pd.Series({ "minutes": s["sum"].sum()}))
b = a.to_dict()['minutes']
c = a['minutes'].sum()
#['INCIDENT_REASON'].value_counts()
b

{'All Others': 11130.0,
 'External': 10047367.29,
 'Fleet': 17768923.7844,
 'Network Management / Other': 12294881.2617,
 'Non-Track Assets': 13056812.5,
 'Operations': 37695543.7765,
 'Severe Weather, Autumn & Structures': 7031243.755000001,
 'Stations': 1889883.9300000002,
 'TOC Other': 9348978.484000001,
 'Track': 5357186.2,
 'Traincrew': 13820856.0484}

In [ ]:
a = incident_data
a['MONTH'] = a['ORIGIN_DEPARTURE_DATE'].str[5:7].astype(int)
a['YEAR'] = a['ORIGIN_DEPARTURE_DATE'].str[0:4].astype(int)

a.groupby(by=['MONTH','YEAR','INCIDENT_REASON']).apply(lambda s: pd.Series({"minutes": s["sum"].sum()})).reset_index()

month_a ={}
for y in ['2019', '2020', '2021']:
  month_a[y] = {}
  for m in range(1,13):
    month_word = datetime.date(int(year),int(month),1).strftime("%B")
    month_a[y][month_word] = 1

SyntaxError: ignored

In [ ]:
#generate stacked bar chart
month_delay.plot(kind='barh', stacked=True)
plt.legend(bbox_to_anchor=(1.0, 1.0))
plt.xlabel("Percent")
plt.ylabel("Year")
plt.title("Distribution of the Delay over the Months for the Years")
plt.savefig(data_path + 'monthlydelay_dist.png', dpi=300, bbox_inches='tight')
plt.show()

In [57]:
import plotly.graph_objects as go

incidents = []
causes = []
count_list = [8.67343869542845e-05/2, 0.07829759590320581/2, 0.29375560470468304/2, 0.07829759590320581/2, 8.67343869542845e-05/2, 0.1384705041578921, 0.04174773233216109, 0.10174974193751472, 0.014727576287808082, 0.0547934067040491, 0.29375560470468304/2, 0.09581212838358216, 0.07285515880130604, 0.10770381640084357]

causes_incident_dict = {
    1 : ('Majeure Event','All Others'),
    2 : ('Majeure Event','External'),
    12 : ('Passengers','Operations'),
    13 : ('Passengers','External'),
    14 : ('Passengers','All Others'),
    3 : ('Equipment','Fleet'),
    4 : ('Equipment','Track'),
    5 : ('Equipment','Non-Track Assets'),
    6 : ('Equipment','Stations'),
    7 : ('Equipment','Weather & Structures'),
    8 : ('Staff','Operations'),
    9 : ('Staff','Network Management'),
    10 : ('Staff','TOC Other'), 
    11 : ('Staff','Traincrew'),
}

for i in causes_incident_dict:
  causes.append(causes_incident_dict[i][0])
  incidents.append(causes_incident_dict[i][1])

fig = go.Figure(go.Parcats(
    dimensions=[
        {'label': 'Cause',
         'values': causes},
        {'label': 'Incident Category',
         'values': incidents}],
   counts= count_list,
))

fig.show()